# 🤖 AgentPro Hiring Assistant

This notebook demonstrates how to build a Hiring Assistant using AgentPro that analyzes candidate profiles, resumes, and GitHub activity to provide detailed assessments for HR professionals.

### Features:
- Resume analysis from Google Drive URLs
- GitHub profile analysis
- Skills and experience matching
- Detailed candidate persona generation
- Company culture fit assessment


## Step 1: Clone AgentPro and Install Dependencies

To get started with **AgentPro**, begin by cloning the official GitHub repository and installing its dependencies.

In [25]:
!git clone https://github.com/traversaal-ai/AgentPro.git
%cd AgentPro
%pip install -r requirements.txt
%pip install gradio PyPDF2 python-docx beautifulsoup4 requests python-dotenv

fatal: destination path 'AgentPro' already exists and is not an empty directory.
/Users/shaikmohammedzubair/Documents/Huzaifa - Work/Hackathon Projects/AgentPro/agentpro/examples/AgentPro/agentpro/AgentPro/agentpro/AgentPro

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Candidate Assessment for Shaik Mohammed Huzaifa**

**1. Skills and Experience Fit**
- **Match to Job Requirements:**
   - **Hard Skills:** The job description seeks a Senior Python Developer with at least 5 years of experience in web development, machine learning, and cloud technologies. Shaik primarily identifies as a frontend developer and has expressed an interest in databases, but lacks explicit experience or proficiency in Python, machine learning, or cloud technologies. This reflects a significant skills gap required for the role.
   - **Soft Skills:** The candidate’s bio highlights a collaborative approach and a passion for community-driven technology, suggesting strengths in teamwork and communication—qualities essential in Agile methods.

**2. GitHub Profile Evaluation**
- **Activity and Contribution Quality:**  
   - The GitHub profile shows no repositories, raising concerns about Shaik's engagement with practical coding and contributions to various projects. This absence s

## Step 2: Set Up API Keys

In [27]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set API keys
os.environ["OPENAI_API_KEY"] = " "
os.environ["TRAVERSAAL_ARES_API_KEY"] = "ares_9607a82a24c329cfb4b0a7b07c0ca178ac699d5fc3b935e0057006cdb6dad790"
os.environ["OPENROUTER_API_KEY"] = "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"

## Step 3: Create the Hiring Agent

In [47]:
from agentpro import AgentPro
from agentpro.tools import AresInternetTool, CodeEngine
import requests
import PyPDF2
import io
import docx
from typing import Dict

class HiringAgent:
    def __init__(self):
        self.agent = AgentPro(tools=[AresInternetTool(), CodeEngine()])

    def extract_text_from_pdf(self, pdf_url: str) -> str:
        response = requests.get(pdf_url)
        pdf_file = io.BytesIO(response.content)
        reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() or "" for page in reader.pages)

    def extract_text_from_docx(self, docx_url: str) -> str:
        response = requests.get(docx_url)
        docx_file = io.BytesIO(response.content)
        doc = docx.Document(docx_file)
        return "\n".join(p.text for p in doc.paragraphs)

    def get_github_readme(self, username: str, repo_name: str) -> str:
        # Try main first, then fallback to master
        for branch in ["main", "master"]:
            readme_url = f"https://raw.githubusercontent.com/{username}/{repo_name}/{branch}/README.md"
            res = requests.get(readme_url)
            if res.status_code == 200:
                return res.text[:1000]
        return "README not available."

    def analyze_github_profile(self, github_url: str) -> Dict:
        username = github_url.rstrip("/").split("/")[-1]

        # Get user info
        user_data = requests.get(f"https://api.github.com/users/{username}").json()

        # Get top 5 public repos
        repos_data = requests.get(f"https://api.github.com/users/{username}/repos").json()
        repos = []
        for repo in repos_data[:5]:
            readme = self.get_github_readme(username, repo["name"])
            repos.append({
                "name": repo["name"],
                "description": repo.get("description", ""),
                "stars": repo.get("stargazers_count", 0),
                "forks": repo.get("forks_count", 0),
                "language": repo.get("language", "N/A"),
                "readme": readme
            })

        return {
            "name": user_data.get("name", username),
            "bio": user_data.get("bio", ""),
            "location": user_data.get("location", ""),
            "public_repos": user_data.get("public_repos", 0),
            "followers": user_data.get("followers", 0),
            "top_repositories": repos
        }

    def analyze_candidate(self, resume_url: str, github_url: str, job_description: str, company_info: str) -> str:
        # Extract resume text
        if resume_url.endswith(".pdf"):
            resume_text = self.extract_text_from_pdf(resume_url)
        elif resume_url.endswith(".docx"):
            resume_text = self.extract_text_from_docx(resume_url)
        else:
            resume_text = "Unsupported resume format."

        # GitHub Profile
        github_data = self.analyze_github_profile(github_url)

        prompt = f"""
You are a senior AI-based hiring assistant. Evaluate this candidate profile comprehensively.

📄 Resume Content:
{resume_text}

👨‍💻 GitHub Profile:
- Name: {github_data['name']}
- Bio: {github_data['bio']}
- Location: {github_data['location']}
- Public Repos: {github_data['public_repos']}
- Followers: {github_data['followers']}

📦 Top Repositories:
{"".join([
    f"""
**{repo['name']}**
- Description: {repo['description']}
- Stars: {repo['stars']}
- Forks: {repo['forks']}
- Language: {repo['language']}
- README Excerpt: {repo['readme'][:300]}...

""" for repo in github_data['top_repositories']
])}

📝 Job Description:
{job_description}

🏢 Company Info:
{company_info}

---

📌 Please assess:
1. Skill & experience match
2. GitHub activity, code quality & practices
3. Technical depth & learning agility
4. Cultural/team fit
5. Strengths & areas to improve
6. Final hiring recommendation

Format response like a well-written internal hiring note.
"""

        data = self.agent(prompt)
        print(f"This is response {data}")
        return {
            "assessment": data
        }

## Step 4: Create Simple Gradio Interface

In [48]:
import gradio as gr

# Initialize the hiring agent
hiring_agent = HiringAgent()

def analyze_candidate(resume_url, github_url, job_description, company_info):
    try:
        result = hiring_agent.analyze_candidate(resume_url, github_url, job_description, company_info)
        return result
    except Exception as e:
        return f"Error analyzing candidate: {str(e)}"

# Create a simple Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🤖 AgentPro Hiring Assistant
    Upload candidate details and get a detailed assessment for your hiring process.
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📝 Candidate Information")
            resume_url = gr.Textbox(
                label="Resume URL",
                placeholder="Enter Google Drive URL of the resume",
                info="Paste the Google Drive URL of the candidate's resume"
            )
            github_url = gr.Textbox(
                label="GitHub Profile",
                placeholder="Enter GitHub profile URL",
                info="Paste the candidate's GitHub profile URL"
            )
            
            gr.Markdown("### 🏢 Job & Company Details")
            job_description = gr.Textbox(
                label="Job Description",
                placeholder="Enter the job description",
                lines=5,
                info="Describe the role and requirements"
            )
            company_info = gr.Textbox(
                label="Company Information",
                placeholder="Enter company details and culture",
                lines=3,
                info="Describe the company culture and environment"
            )
            
            analyze_btn = gr.Button("Analyze Candidate", variant="primary")
        
        with gr.Column(scale=2):
            gr.Markdown("### 📊 Assessment Results")
            output = gr.Textbox(
                label="Candidate Assessment",
                lines=20,
                info="Detailed analysis of the candidate's profile"
            )

        with gr.Column(scale=2):
            gr.Markdown("### 📊 Assessment Results")
            output = gr.Markdown()
    
    # Add example inputs
    gr.Markdown("### 💡 Example Inputs")
    gr.Examples(
        examples=[
            [
                "https://drive.google.com/example-resume.pdf",
                "https://github.com/example-user",
                "Looking for a Senior Python Developer with 5+ years of experience in web development, machine learning, and cloud technologies. Must have strong problem-solving skills and experience with agile methodologies.",
                "Tech startup focused on AI solutions, fast-paced environment, collaborative culture, emphasis on innovation and continuous learning."
            ]
        ],
        inputs=[resume_url, github_url, job_description, company_info],
        outputs=[output],
        fn=analyze_candidate,
        cache_examples=True
    )
    
    # Add footer
    gr.Markdown("""
    ---
    *Powered by AgentPro - An AI-powered hiring assistant*
    """)
    
    # Connect the analyze button
    analyze_btn.click(
        fn=analyze_candidate,
        inputs=[resume_url, github_url, job_description, company_info],
        outputs=[output]
    )

# Launch the app
app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7879
Caching examples at: '/Users/shaikmohammedzubair/Documents/Huzaifa - Work/Hackathon Projects/AgentPro/agentpro/examples/AgentPro/agentpro/AgentPro/agentpro/AgentPro/.gradio/cached_examples/298'
* Running on public URL: https://622e9409ccb2f195d7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Internal Hiring Note: Candidate Evaluation for Shaik Mohammed Huzaifa

**Candidate Overview:**
Shaik Mohammed Huzaifa is being evaluated for the Full Stack Python position. His GitHub profile indicates active engagement with both backend and frontend technologies; however, the absence of a structured resume limits our ability to assess his qualifications comprehensively.

---

**1. Skill & Experience Match:**
Shaik demonstrates familiarity with key technologies necessary for the role, particularly Python for backend development and JavaScript/React for frontend tasks. While he possesses foundational skills relevant to full-stack development, the lack of a detailed resume complicates the evaluation of his practical experience and depth of expertise in related projects.

**2. GitHub Activity, Code Quality, and Practices:**
Shaik has significant activity on GitHub, with 86 public repositories reflecting a strong commitment to coding and software development. However, many repositories